In [ ]:
import torch
from tensordict import TensorDict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
from cycler import cycler
import os

# colors = [(0.0, 0.588, 0.509), (0.635, 0.133, 0.137), (0.137, 0.631, 0.878)]
# plt.rcParams['axes.prop_cycle'] = cycler(color=colors)

output_path = "./figures"
os.makedirs(output_path, exist_ok=True)

#  SMARD DE/AT/LU

In [ ]:
file_path = '0_raw/Gro_handelspreise_201507010000_201807010000_Stunde.csv'
raw_df = pd.read_csv(file_path, sep=';', header=0, converters={'DE/AT/LU [€/MWh] Originalauflösungen': lambda x: float(x.replace(",", "."))})
raw_values = raw_df['DE/AT/LU [€/MWh] Originalauflösungen']
timestep_raw_values = pd.Series([x/100 for x in raw_values for _ in range(2)]).round(4)

start = datetime(2015, 7, 1, 0, 30)
end = datetime(2018, 7, 1, 0, 0)
dates = pd.date_range(start, end, freq='30min')

price_data = pd.DataFrame({'Date': dates, 'Price [$ per kW/h]': timestep_raw_values}, columns=['Date', 'Price [$ per kW/h]'])

In [ ]:


plt.style.use('seaborn-v0_8-whitegrid')

textwidth_pt = 432.4819
inches_per_pt = 1 / 72.27
fig_width = textwidth_pt * inches_per_pt      # 6.3 inches
aspect_ratio = 0.6                            # height = 60% of width
fig_height = fig_width * aspect_ratio

fig, ax = plt.subplots(figsize=(fig_width, fig_height))
ax.plot(price_data['Date'], price_data['Price [$ per kW/h]'], linewidth=2)

ax.axvline(datetime(2016,7,1), linestyle='--', color='red')
ax.axvline(datetime(2017,7,1), linestyle='--', color='red')

# Tick format like "%b\n%Y"
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=4))  # every 3 months
ax.tick_params(axis='x', labelsize=10)

ax.tick_params(axis='x', which='major', pad=5) 
# ax.tick_params(axis='y', which='major', pad=8)
ax.xaxis.labelpad = 10                           
ax.yaxis.labelpad = 10


# ax.set_title("Price over all years", 
# 						 fontweight='bold',
# 						 fontsize=13,
# 							 pad=10)  
ax.set_xlabel("Date", fontsize=11)
ax.set_ylabel("Price [$ per kW/h]", fontsize=11)
ax.set_xlim(price_data['Date'].iloc[0], price_data['Date'].iloc[-1])

# fig.autofmt_xdate()
plt.tight_layout()
plt.savefig(f"{output_path}/price_over_all_years.pdf", bbox_inches='tight')  # high-quality export
plt.show()

In [ ]:
first_year = price_data[price_data['Date']<datetime(2016,7,1,0,30)]
mean_first_year = torch.tensor(first_year['Price [$ per kW/h]'].to_numpy(dtype="float32")).unfold(dimension=0,size=48,step=48).mean(0)

second_year = price_data[(price_data['Date']>=datetime(2016,7,1,0,30)) & (price_data['Date']<datetime(2017,7,1,0,30))]
mean_second_year = torch.tensor(second_year['Price [$ per kW/h]'].to_numpy(dtype="float32")).unfold(dimension=0,size=48,step=48).mean(0)

third_year= price_data[(price_data['Date']>=datetime(2017,7,1,0,30)) & (price_data['Date']<datetime(2018,7,1,0,30))]
mean_third_year = torch.tensor(third_year['Price [$ per kW/h]'].to_numpy(dtype="float32")).unfold(dimension=0,size=48,step=48).mean(0)
df = pd.DataFrame({
	'Time': pd.date_range(start="2000-01-01 00:30", end="2000-01-02 00:00", freq="30min"),
	'First year': mean_first_year,
	'Second year': mean_second_year,
	'Third year': mean_third_year
})

fig, ax = plt.subplots(figsize=(fig_width, fig_height))
ax.plot(df['Time'] , df['First year'], linewidth=2, label='First year')
ax.plot(df['Time'] , df['Second year'], linewidth=2, label='Second year')
ax.plot(df['Time'] , df['Third year'], linewidth=2, label='Third year')

ax.xaxis.set_major_locator(mdates.HourLocator(interval=3))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))

ax.tick_params(axis='x', which='major', pad=8) 
ax.tick_params(axis='y', which='major', pad=5)
ax.xaxis.labelpad = 10                           
ax.yaxis.labelpad = 10
ax.set_xlim(df['Time'].iloc[0], df['Time'].iloc[-1])

# ax.set_title("Mean hour price over each years", 
# 			 fontweight='bold',
# 			 fontsize=13,
# 			   pad=10)  
ax.set_xlabel("Time", fontsize=11)
ax.set_ylabel("Price [$ per kW/h]", fontsize=11)

ax.legend(
	loc='upper left',
	facecolor='white',
	frameon=True,
	fontsize=9)
plt.tight_layout()
plt.savefig(f"{output_path}/price_mean_over_years.pdf", bbox_inches='tight') 
plt.show()

# Ausgrid

In [ ]:
building_ids = [13,20,33,35,74,75,82,87,88,101,106,109,130,144,152,153,157,161,169,176]
# building_ids = [88,101]
# building_ids = [13,20]

In [ ]:
file_paths = [
	'0_raw/2010-2011 Solar home electricity data.csv',
	'0_raw/2011-2012 Solar home electricity data.csv',
	'0_raw/2012-2013 Solar home electricity data.csv']

dataframe = pd.DataFrame()

for file_path in file_paths:
	df = pd.read_csv(file_path, header=1)

	if '2010-2011' in file_path:
		df.date = pd.to_datetime(df.date, format='%d-%b-%y')
	else:
		df.date = pd.to_datetime(df.date, format='%d/%m/%Y')
	
	if 'Row Quality' in df.columns:
		df.drop('Row Quality', axis=1, inplace=True)
	df.drop('Generator Capacity', axis=1, inplace=True)
	df.drop('Postcode', axis=1, inplace=True)

	# Append the modified dataframe to the list
	dataframe = pd.concat([dataframe, df], ignore_index=True)
	
pv = []
load = []
for building_id in building_ids:
	df = dataframe[dataframe['Customer'] == building_id]
	
	df_long = df.melt(
	id_vars=["Customer", "Consumption Category", "date"],
	var_name="time",
	value_name="value"
	).replace('0:00', '24:00')
	# Step 2: Combine date and time into one datetime column
	df_long["datetime"] = pd.to_datetime(df_long["date"]) + pd.to_timedelta(df_long["time"] + ":00")

	# Step 3: Pivot so that categories become columns
	df_pivot = df_long.pivot_table(
		index=["Customer", "datetime"],
		columns="Consumption Category",
		values="value"
	).reset_index()
	
	start = datetime(2010, 7, 1, 0, 30)
	end = datetime(2013, 7, 1, 0, 0)
	dates = pd.date_range(start, end, freq='30min')

	df_full = (df_pivot.set_index("datetime")
			 .reindex(dates)          # introduce missing rows
			 .rename_axis("datetime")
			 .reset_index()).drop(columns=['Customer'])
	df_full.index.name = None
	df_full.columns.name = None
	if 'CL' in df.columns:
		df_full['Consumption'] = (df_full['CL'] + df_full['GC']).round(3)
	else:
		df_full['Consumption'] = df_full['GC']
	df_full['Prosumption'] = (df_full['Consumption'] - df_full['GG']).round(3)


	first_year = df_full[df_full['datetime']<datetime(2011,7,1,0,30)]
	mean_first_year = torch.tensor(first_year['GG'].to_numpy(dtype="float32")).unfold(dimension=0,size=48,step=48).mean(0)

	second_year = df_full[(df_full['datetime']>=datetime(2011,7,1,0,30)) & (df_full['datetime']<datetime(2012,7,1,0,30))]
	mean_second_year = torch.tensor(second_year['GG'].to_numpy(dtype="float32")).unfold(dimension=0,size=48,step=48).mean(0)

	third_year= df_full[(df_full['datetime']>=datetime(2012,7,1,0,30)) & (df_full['datetime']<datetime(2013,7,1,0,30))]
	mean_third_year = torch.tensor(third_year['GG'].to_numpy(dtype="float32")).unfold(dimension=0,size=48,step=48).mean(0)

	pv.append(torch.sum(mean_first_year+mean_second_year+mean_third_year)*0.5)

	df = pd.DataFrame({
		'Time': pd.date_range(start="2000-01-01 00:30", end="2000-01-02 00:00", freq="30min"),
		'First year': mean_first_year,
		'Second year': mean_second_year,
		'Third year': mean_third_year
	})

	fig, ax = plt.subplots(figsize=(fig_width, fig_height))
	ax.plot(df['Time'] , df['First year'], linewidth=2, label='First year')
	ax.plot(df['Time'] , df['Second year'], linewidth=2, label='Second year')
	ax.plot(df['Time'] , df['Third year'], linewidth=2, label='Third year')
	
	ax.xaxis.set_major_locator(mdates.HourLocator(interval=3))
	ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))

	ax.tick_params(axis='x', which='major', pad=8) 
	ax.tick_params(axis='y', which='major', pad=5)
	ax.xaxis.labelpad = 10                           
	ax.yaxis.labelpad = 10
	ax.set_xlim(df['Time'].iloc[0], df['Time'].iloc[-1])

	# ax.set_title(f"Mean hour PV Generation over each years for building {building_id}", 
	# 			fontweight='bold',
	# 			fontsize=13,
	# 			pad=10)  
	ax.set_xlabel("Time", fontsize=11)
	ax.set_ylabel("PV Generation [kW]", fontsize=11)

	ax.legend(
		loc='upper left',
		facecolor='white',
		frameon=True,
		fontsize=9)
	plt.tight_layout()
	plt.savefig(f"{output_path}/{building_id}_pv_mean_over_years.pdf", bbox_inches='tight') 
	plt.show()

	first_year = df_full[df_full['datetime']<datetime(2011,7,1,0,30)]
	mean_first_year = torch.tensor(first_year['Consumption'].to_numpy(dtype="float32")).unfold(dimension=0,size=48,step=48).mean(0)

	second_year = df_full[(df_full['datetime']>=datetime(2011,7,1,0,30)) & (df_full['datetime']<datetime(2012,7,1,0,30))]
	mean_second_year = torch.tensor(second_year['Consumption'].to_numpy(dtype="float32")).unfold(dimension=0,size=48,step=48).mean(0)

	third_year= df_full[(df_full['datetime']>=datetime(2012,7,1,0,30)) & (df_full['datetime']<datetime(2013,7,1,0,30))]
	mean_third_year = torch.tensor(third_year['Consumption'].to_numpy(dtype="float32")).unfold(dimension=0,size=48,step=48).mean(0)

	load.append(torch.sum(mean_first_year+mean_second_year+mean_third_year)*0.5)

	df = pd.DataFrame({
		'Time': pd.date_range(start="2000-01-01 00:30", end="2000-01-02 00:00", freq="30min"),
		'First year': mean_first_year,
		'Second year': mean_second_year,
		'Third year': mean_third_year
	})

	# df = df.melt('Time', var_name='Year', value_name='Consumption')
	fig, ax = plt.subplots(figsize=(fig_width, fig_height))
	ax.plot(df['Time'] , df['First year'], linewidth=2, label='First year')
	ax.plot(df['Time'] , df['Second year'], linewidth=2, label='Second year')
	ax.plot(df['Time'] , df['Third year'], linewidth=2, label='Third year')

	ax.xaxis.set_major_locator(mdates.HourLocator(interval=3))
	ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))

	ax.tick_params(axis='x', which='major', pad=8) 
	ax.tick_params(axis='y', which='major', pad=5)
	ax.xaxis.labelpad = 10                           
	ax.yaxis.labelpad = 10
	ax.set_xlim(df['Time'].iloc[0], df['Time'].iloc[-1])

	# ax.set_title(f"Mean hour Load over each years for building {building_id}", 
	# 			fontweight='bold',
	# 			fontsize=13,
	# 			pad=10)  
	ax.set_xlabel("Time", fontsize=11)
	ax.set_ylabel("Load [kW]", fontsize=11)

	ax.legend(
		loc='upper left',
		facecolor='white',
		frameon=True,
		fontsize=9)
	plt.tight_layout()
	plt.savefig(f"{output_path}/{building_id}_load_mean_over_years.pdf", bbox_inches='tight') 
	plt.show()

In [ ]:
colors = [
    'red', 'blue', 'green', 'orange', 'purple',
    'cyan', 'magenta', 'yellow', 'black', 'brown',
    'pink', 'gray', 'olive', 'navy', 'teal',
    'gold', 'maroon', 'lime', 'indigo', 'turquoise'
]

In [ ]:
for i in range(20):
    id = building_ids[i]
    plt.scatter(load[i],pv[i], label=f'Building {id}', color=colors[i])

plt.xlabel('Mean Load [kWh per day]')          # X-axis label
plt.ylabel('Mean PV Generation [kWh per day]')     # Y-axis label
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.savefig(f"{output_path}/scatter_load_pv.pdf", bbox_inches='tight') 
plt.show()